In [1]:
from tqdm import tqdm

In [2]:
from collections import defaultdict

In [8]:
import os

In [3]:
def read_run(path):
    run = {}
    with open(path) as f:
        for line in tqdm(f):
            qid, _ , did, _ , score ,_ = line.strip().split()
            score = float(score)
            run.setdefault(qid, {})
            run[qid][did] = score
    return run

In [14]:
rerank_run = read_run("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs/rus_ar_rerank_dpr_AUTO_colbert_query_maxlen_32.run")

baseline_run = read_run("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs/round_robin_ar_ru_baseline.run")
        
baseline_list = {}
for qid in baseline_run:
    ranklist = list(sorted(baseline_run[qid].items(), key=lambda x: (x[1], x[0]), reverse=True))
    baseline_list.setdefault(qid, [])
    for retdoc in ranklist:
        baseline_list[qid].append(retdoc[0])

54000it [00:00, 494469.86it/s]
54000it [00:00, 892426.20it/s]


In [15]:
rrf_k = 60

In [16]:
fusion_run = defaultdict(dict)
for qid in tqdm(rerank_run):
    rerank_list = list(sorted(rerank_run[qid].items(), key=lambda x: (x[1], x[0]), reverse=True))
    for i in range(len(rerank_list)):
        did = rerank_list[i][0]
        rerank_at = i + 1
        baseline_at = baseline_list[qid].index(did) + 1
        # print(rerank_at, baseline_at)
        fused_score = (1/(rrf_k + rerank_at)) + (1/(rrf_k + baseline_at))
        fusion_run[qid][did] = fused_score

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:00<00:00, 96.88it/s]


In [17]:
fusion_file = os.path.join("/work/snaseri_umass_edu/better_P3/experiments/BetterRerankerS1/runs", f'fusion_ru_ar_reranker_dp_and_ru_ar_baseline_rrf_{rrf_k}.run')
with open(fusion_file, 'wt') as runfile:
    for qid in fusion_run:
        dummy_score = 100000
        fused_scores = list(sorted(fusion_run[qid].items(), key=lambda x: (x[1], x[0]), reverse=True))
        for i, (did, score) in enumerate(fused_scores):
            runfile.write(f'{qid} 0 {did} {i+1} {dummy_score} run\n')
            dummy_score -= 1